## Формирование обучающей и тестовой выборок (n-pair подход)

В этом ноутбуке мы формируем датасеты для обучения моделей на основе парных взаимодействий пользователей с товарами:

- `df_train_baseline_0_n_pair.csv` — сбалансированная выборка позитивных и негативных примеров для baseline-0 модели, сгенерированная подходом n-pair;
- `df_train_baseline_1_n_pair.csv` — сбалансированная выборка позитивных и негативных примеров для baseline-1 модели, сгенерированная подходом n-pair;
- `df_train_CLIP_n_pair.csv` — сбалансированная выборка позитивных и негативных примеров для мэтчинговой модел, сгенерированная подходом n-pair;
- `df_test_ground_truth_n_pair.csv` — тестовый лог взаимодействий за выбранный временной период (реальные положительные примеры).

Также добавляется словарь `ground_truth`, необходимый для расчёта метрик `Precision@K`, `MAP@K`.


In [ ]:
# Импорт требуемых библиотек
import pandas as pd
import datetime
import numpy as np

In [2]:
# Подгрузим датасет с отзывами
df = pd.read_csv("data/amazon_interactions_filtered.csv")

In [ ]:
# Определим период, за который берем выборку
# а именно, согласно анализу файла по взаимодействиям юзеров
# с товарами, мы решили, что возьмем выборку за весь 2016 год
year = 2016
start = int(datetime.datetime(year, 1, 1).timestamp())
end = int(datetime.datetime(year, 12, 31, 23, 59, 59).timestamp())

In [ ]:
# Фильтрация выборки по году
df = df[(df["timestamp"] >= start) & (df["timestamp"] <= end)].copy()

In [5]:
from sklearn.model_selection import train_test_split

# Шаффлим и делим случайно на train/test
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42, shuffle=True)

In [6]:
print(f"Train: {len(df_train):,} строк, {df_train['user_id'].nunique():,} пользователей, {df_train['item_id'].nunique():,} товаров")
print(f"Test:  {len(df_test):,} строк, {df_test['user_id'].nunique():,} пользователей, {df_test['item_id'].nunique():,} товаров")

Train: 608,892 строк, 294,160 пользователей, 72,132 товаров
Test:  67,655 строк, 58,977 пользователей, 26,490 товаров


In [ ]:
# Выделим уникальных юзеров на трейне и тесте
users_train = set(df_train["user_id"])
users_test = set(df_test["user_id"])

# Сколько юзеров в test уже встречались в train
overlap_users = users_test & users_train

In [8]:
print(f"Всего пользователей в test: {len(users_test):,}")
print(f"Из них уже были в train: {len(overlap_users):,}")
print(f"Доля покрытых пользователей: {round(len(overlap_users) / len(users_test) * 100, 2)}%")

Всего пользователей в test: 58,977
Из них уже были в train: 44,348
Доля покрытых пользователей: 75.2%


In [ ]:
# Оставим в тестовой выборке только тех юзеров, кто есть в train
df_test = df_test[df_test["user_id"].isin(users_train)].copy()
print(f"Тест после фильтрации: {len(df_test):,} строк, {df_test['user_id'].nunique():,} пользователей")

Тест после фильтрации: 51,938 строк, 44,348 пользователей


### Генерация негативных примеров для обучения модели

В исходных данных `train` представлены только **позитивные взаимодействия**: пользователи оставили отзывы на товары, что означает факт взаимодействия или покупки. Однако, для обучения модели ранжирования нам необходимы также **негативные примеры** — товары, с которыми пользователь **не взаимодействовал**.

#### Что делаем:

- Для каждого положительного примера (`user_id`, `item_id`, `label=1`) мы сгенерируем 1 негативный пример (`user_id`, `item_id`, `label=0`)
- Негативный товар выбирается **случайным образом** из числа тех, с которыми пользователь **не взаимодействовал** в `train`
- Мы формируем итоговую обучающую выборку с бинарной меткой `label ∈ {0, 1}`

#### Ограничения:

- Негативные товары отбираются **только из тех, которые присутствуют в `train`**
- Мы исключаем из выборки те пары, которые могли бы быть положительными (по известным взаимодействиям)

#### Результат:

- Обучающая выборка сбалансирована (`1:1 sampling`)
- Можно использовать в классификаторе (CatBoost) или для расчёта вероятностей `predict_proba - ранжирование`
- Данные готовы к дальнейшей джойну с мета-информацией



In [ ]:
import random
from tqdm import tqdm
import sys
import os


disable_tqdm = os.getenv("TQDM_DISABLE", "0") == "1"

In [11]:
# Все уникальные товары из train
all_items = df_train["item_id"].unique().tolist()

In [ ]:
# Построим маппинг: user_id - set(item_id), с которыми он взаимодействовал
user_pos_items = df_train.groupby("user_id")["item_id"].apply(set).to_dict()

In [ ]:
# Список для хранения новых примеров сгенерированной
# негативной выборки
neg_samples = []

In [ ]:
# В цикле по каждому юзеру сгенерим негативные примеры
for user, pos_items in tqdm(user_pos_items.items(), desc="Генерация негативных примеров", file=sys.stdout):
    for pos_item in pos_items:
        while True:
            neg_item = random.choice(all_items)
            if neg_item not in pos_items:
                neg_samples.append((user, neg_item, 0))  # негативка
                break

Генерация негативных примеров: 100%|██████████| 294160/294160 [00:00<00:00, 539936.02it/s]


In [15]:
# Создаём датафрейм с негативами
df_neg = pd.DataFrame(neg_samples, columns=["user_id", "item_id", "label"])

In [16]:
# Позитивные примеры
df_pos = df_train[["user_id", "item_id"]].copy()
df_pos["label"] = 1

In [17]:
# Финальный train dataset
df_train_final = pd.concat([df_pos, df_neg], ignore_index=True)

In [18]:
df_train_final.head()

,user_id,item_id,label
0,A353D8STHBQZKY,B0055MGVO2,1
1,A1XQ84TW915PLS,B00H4HKNA4,1
2,A1QU9S2PKRX8R0,B011397NIS,1
3,A1NUWNKPW2NGYD,B000H225TU,1
4,A1A7NBKZZQ4Q07,B000MT22QE,1


In [ ]:
# Подгрузим датасет со сгенерированными фичами товаров
df_meta = pd.read_csv(
    "data/amazon_meta_clean.csv",
    na_values=[""],  # исключаем "Unknown"
    keep_default_na=False
)

In [20]:
print("Проверим пропуски после загрузки:")
print(df_meta[["brand", "category_main", "price_clean"]].isna().sum())

Проверим пропуски после загрузки:
brand            0
category_main    0
price_clean      0
dtype: int64


In [21]:
# Удаляем CLIP-фичи
clip_cols = [col for col in df_meta.columns if col.startswith("clip_text_") or col.startswith("clip_img_")]
df_meta = df_meta.drop(columns=clip_cols)

In [22]:
print(df_meta.shape)
df_meta.head(2)

(148948, 112)


,asin,title_len,title_has_digit,description_text_len,text_full,brand,is_top20_brand,price_clean,has_price,category_main,...,tfidf_91,tfidf_92,tfidf_93,tfidf_94,tfidf_95,tfidf_96,tfidf_97,tfidf_98,tfidf_99,tfidf_100
0,0853347867,-1.251232,0,-0.631098,"Trim Healthy Mama Xylitol. Shipped from UK, pl...",Unknown,1,-0.518064,0,Cooking & Baking,...,1.099800,0.677096,1.538071,-0.064891,-0.611596,2.097358,-1.535881,-0.427645,1.433121,0.793126
1,4639725043,-0.735001,1,-0.426547,Lipton Yellow Label Tea (loose tea) - 450g. Li...,Lipton,0,0.061373,1,Beverages,...,-0.016708,0.899488,-0.328298,1.685778,0.418503,1.372579,-0.176657,-0.481735,-1.121662,0.877780


In [ ]:
# Проверим, что названия колонок совпадают
print("asin" in df_meta.columns)
print("item_id" in df_train_final.columns)

True
True


In [ ]:
# Переименуем asin в item_id
df_meta.rename(columns={"asin": "item_id"}, inplace=True)

### Для baseline 0

In [ ]:
# Мерджим трейн с фичами товаров
df_train_merged = df_train_final.merge(
    df_meta,
    on="item_id",
    how="left" # сохраняем все примеры, даже если нет мета-данных
)

In [26]:
print(df_train_merged.shape)
print(df_train_merged["label"].value_counts())
print("Доля строк с отсутствующей мета-информацией:", df_train_merged.isna().mean().round(3))

(1190803, 114)
label
1    608892
0    581911
Name: count, dtype: int64
Доля строк с отсутствующей мета-информацией: user_id            0.000
item_id            0.000
label              0.000
title_len          0.195
title_has_digit    0.195
                   ...  
tfidf_96           0.195
tfidf_97           0.195
tfidf_98           0.195
tfidf_99           0.195
tfidf_100          0.195
Length: 114, dtype: float64


In [ ]:
# Отфильтруем данные с пропусками в нижеупомянутых полях
required_fields = ["text_full", "image_main", "brand"]

df_train_clean = df_train_merged.dropna(subset=required_fields).copy()
print(f"После удаления строк без критичных признаков: {len(df_train_clean):,} строк")

После удаления строк без критичных признаков: 958,963 строк


In [28]:
# Создадим ground truth - словарь, где:
# ключ — user_id
# значение — множество товаров, с которыми он взаимодействовал в test
ground_truth = df_test.groupby("user_id")["item_id"].apply(set).to_dict()

### Ground truth и offline-оценка рекомендательной системы

После формирования обучающей выборки (`df_train_final`) и обучения модели мы хотим проверить, насколько качественно она ранжирует товары для каждого пользователя.

Для этого мы используем `test`-датасет, собранный **по временной отсечке**. Он содержит **реальные взаимодействия**, которые произошли после `train`-периода. Такие взаимодействия мы считаем "золотым стандартом" — **ground truth**.

#### Что мы делаем:

- Сгруппируем `df_test` по пользователю и сохраним все `item_id`, с которыми он взаимодействовал
- Для каждого `user_id` модель будет генерировать top-k рекомендаций
- Мы сравним рекомендации модели с тем, что действительно произошло


In [29]:
df_train_clean.head(1)

,user_id,item_id,label,title_len,title_has_digit,description_text_len,text_full,brand,is_top20_brand,price_clean,...,tfidf_91,tfidf_92,tfidf_93,tfidf_94,tfidf_95,tfidf_96,tfidf_97,tfidf_98,tfidf_99,tfidf_100
0,A353D8STHBQZKY,B0055MGVO2,1,1.208457,1.0,0.465168,"Crystal Light Drink Mix, Decaf Lemon Iced Tea,...",Crystal Light,0.0,0.861651,...,-0.471058,3.841364,2.462334,-0.442178,3.08271,-3.350286,1.085773,3.615367,2.13126,-1.428875


In [ ]:
# Сохраняем данные

# Тренировочная выборка с мета-признаками
df_train_clean.to_csv("data/df_train_baseline_0_n_pair.csv", index=False)

# Тест как лог взаимодействий (без товарных фичей, без негативов)
# ground_truth можно будет пересчитать по df_test в следующем ноутбуке
df_test.to_csv("data/df_test_ground_truth_n_pair.csv", index=False)

### baseline 1

In [31]:
# Загружаем мета-датасет с уже готовыми tfidf_фичами
df_meta_tfidf = pd.read_csv(
    "data/amazon_meta_clean.csv",
    na_values=[""],
    keep_default_na=False
)

In [ ]:
# Переименуем asin в item_id
df_meta_tfidf.rename(columns={"asin": "item_id"}, inplace=True)

In [33]:
# Удалим clip-эмбеддинги (они здесь не нужны)
clip_cols = [col for col in df_meta_tfidf.columns if col.startswith("clip_text_") or col.startswith("clip_img_")]
df_meta_tfidf = df_meta_tfidf.drop(columns=clip_cols)

In [ ]:
# Мерджим трейн с фичами товаров после tf-idf
# для baseline 1
df_train_merged = df_train_final.merge(
    df_meta_tfidf,
    on="item_id",
    how="left"
)

In [35]:
print(df_train_merged.shape)
print(df_train_merged["label"].value_counts())
print("Доля строк с отсутствующей мета-информацией:", df_train_merged.isna().mean().round(3))

(1190803, 114)
label
1    608892
0    581911
Name: count, dtype: int64
Доля строк с отсутствующей мета-информацией: user_id            0.000
item_id            0.000
label              0.000
title_len          0.195
title_has_digit    0.195
                   ...  
tfidf_96           0.195
tfidf_97           0.195
tfidf_98           0.195
tfidf_99           0.195
tfidf_100          0.195
Length: 114, dtype: float64


In [ ]:
# Отфильтруем данные с пропусками в нижеупомянутых полях
required_fields = ["text_full", "image_main", "brand"]

df_train_clean = df_train_merged.dropna(subset=required_fields).copy()
print(f"После удаления строк без критичных признаков: {len(df_train_clean):,} строк")

После удаления строк без критичных признаков: 958,963 строк


In [ ]:
# Удалим уже не нужные поля text_full и image_main
df_train_clean.drop(columns=["text_full", "image_main"], inplace=True)

In [ ]:
# Выведем итоговый список полей
df_train_clean.columns.tolist()

['user_id',
 'item_id',
 'label',
 'title_len',
 'title_has_digit',
 'description_text_len',
 'brand',
 'is_top20_brand',
 'price_clean',
 'has_price',
 'category_main',
 'is_top9_category_main',
 'tfidf_1',
 'tfidf_2',
 'tfidf_3',
 'tfidf_4',
 'tfidf_5',
 'tfidf_6',
 'tfidf_7',
 'tfidf_8',
 'tfidf_9',
 'tfidf_10',
 'tfidf_11',
 'tfidf_12',
 'tfidf_13',
 'tfidf_14',
 'tfidf_15',
 'tfidf_16',
 'tfidf_17',
 'tfidf_18',
 'tfidf_19',
 'tfidf_20',
 'tfidf_21',
 'tfidf_22',
 'tfidf_23',
 'tfidf_24',
 'tfidf_25',
 'tfidf_26',
 'tfidf_27',
 'tfidf_28',
 'tfidf_29',
 'tfidf_30',
 'tfidf_31',
 'tfidf_32',
 'tfidf_33',
 'tfidf_34',
 'tfidf_35',
 'tfidf_36',
 'tfidf_37',
 'tfidf_38',
 'tfidf_39',
 'tfidf_40',
 'tfidf_41',
 'tfidf_42',
 'tfidf_43',
 'tfidf_44',
 'tfidf_45',
 'tfidf_46',
 'tfidf_47',
 'tfidf_48',
 'tfidf_49',
 'tfidf_50',
 'tfidf_51',
 'tfidf_52',
 'tfidf_53',
 'tfidf_54',
 'tfidf_55',
 'tfidf_56',
 'tfidf_57',
 'tfidf_58',
 'tfidf_59',
 'tfidf_60',
 'tfidf_61',
 'tfidf_62',
 'tfidf

In [ ]:
# Вторая тренировочная выборка для baseline 1
# с признаками товаров после tf-idf
df_train_clean.to_csv("data/df_train_baseline_1_n_pair.csv", index=False)

### CLIP-эмбеддинги (готовые)

In [40]:
# Подгружаем эмбеддинги и мета-фичи
df_meta_clip = pd.read_csv(
    "data/amazon_meta_clean.csv",
    na_values=[""],
    keep_default_na=False
)

In [ ]:
# Переименуем asin в item_id
df_meta_clip.rename(columns={"asin": "item_id"}, inplace=True)

In [42]:
# Удалим tfidf-фичи (они здесь не нужны)
tfidf_cols = [col for col in df_meta_clip.columns if col.startswith("tfidf_")]
df_meta_clip = df_meta_clip.drop(columns=tfidf_cols)

In [ ]:
# Мерджим трейн с фичами товаров после CLIP-энкодера
# для мэтчинговой модели
df_train_merged_clip = df_train_final.merge(
    df_meta_clip,
    on="item_id",
    how="left"
)

In [ ]:
# Отфильтруем данные с пропусками в нижеупомянутых полях
required_fields = ["text_full", "image_main", "brand"]
df_train_clip_clean = df_train_merged_clip.dropna(subset=required_fields).copy()
print(f"После удаления строк без критичных признаков: {len(df_train_clip_clean):,} строк")

После удаления строк без критичных признаков: 958,963 строк


In [45]:
# Удалим текст и картинки (они уже в эмбеддингах)
df_train_clip_clean.drop(columns=["text_full", "image_main"], inplace=True)

In [46]:
# Сохраняем CLIP-трейн
df_train_clip_clean.to_csv("data/df_train_CLIP_n_pair.csv", index=False)

### Выводы и сохранение данных

- Итоговые обучающие выборки (`df_train_baseline_0_n_pair.csv`, `df_train_baseline_1_n_pair.csv`, `df_train_CLIP_n_pair.csv`) содержат сбалансированные пары `(user, item)` с метками 0 и 1 (проставленных на основе n-pair подхода).
- Тестовая выборка (`df_test_ground_truth_n_pair.csv`) представляет собой лог реальных взаимодействий за выбранный год, на основе которого будет построен `ground_truth` — словарь соответствия `user_id → set(item_id)`, используемый для оценки рекомендаций.

Эти данные будут использоваться в ноутбуке с обучением и валидацией моделей.
